<a href="https://colab.research.google.com/github/Medissaoui07/Reinforcement-Learning-with-TorchRL/blob/main/RL(DQN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchrl
!pip install tensordict
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.9/359.9 kB 29.8 MB/s eta 0:00:00


In [ ]:
from torchrl.envs import GymEnv , TransformedEnv , StepCounter

env=TransformedEnv(GymEnv('CartPole') , StepCounter() )

from tensordict.nn import TensorDictModule as Mod, TensorDictSequential as Seq


In [ ]:
from torchrl.modules import EGreedyModule , MLP , QValueModule

value_module=MLP(out_features=env.action_spec.shape[-1],num_cells=[64,64])
value_net=Mod(value_module,in_keys=["observation"], out_keys=["action_value"])
policy=Seq(value_net , QValueModule(spec=env.action_spec))

exploration_module=EGreedyModule(env.action_spec,annealing_num_steps=1000,eps_init=0.9)

epsilon_policy=Seq(policy , exploration_module)

In [ ]:
from torchrl.collectors import SyncDataCollector
from torchrl.data.replay_buffers import ReplayBuffer , LazyTensorStorage

frames_per_batch=100

data_collector=SyncDataCollector(env,epsilon_policy ,total_frames=-1 , frames_per_batch=100)
replay_buffer=ReplayBuffer(storage = LazyTensorStorage(10000))


In [ ]:
from torch.optim import Adam
from torchrl.objectives import DQNLoss , SoftUpdate

loss=DQNLoss(value_network=policy , action_space=env.action_spec )
optimizer=Adam(loss.parameters(),lr=0.001)
updater=SoftUpdate(loss, eps=0.99)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from torchrl._utils import logger
from torchrl.record import VideoRecorder , CSVLogger
path='./training'
Logger=CSVLogger("DQN" , log_dir=path , video_format="mp4")

video_recorder=VideoRecorder(Logger , tag="video")

record_env=TransformedEnv(GymEnv('CartPole' , from_pixels=True , pixels_only=False)  , video_recorder ,
                          )



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/

In [ ]:
batch_size = 100
num_steps = 1000
total_count = 0
total_episodes = 0
import time

for i, data in enumerate(data_collector):
    print(f"Iteration: {i}")
    replay_buffer.extend(data)
    print(f"Replay buffer size: {len(replay_buffer)}")

    if len(replay_buffer) < num_steps:
        continue

    sample = replay_buffer.sample(batch_size)
    print(f"Sample keys: {sample.keys()}")

    # Measure loss computation time
    start_time = time.time()
    loss_values = loss(sample)
    print(f"Loss computation time: {time.time() - start_time:.2f}s")

    loss_values["loss"].backward()
    optimizer.step()
    optimizer.zero_grad()

    exploration_module.step()
    updater.step()

    if i % 10 == 0:
        print(f"Replay buffer length: {len(replay_buffer)}")

    total_count += data.numel()
    total_episodes += data["next", "done"].sum()

    # Early stopping condition (example)
    if total_episodes >= 1000:
        print("Training completed!")
        break

print(f"{total_count} steps ; {total_episodes} episodes")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Iteration: 0
Replay buffer size: 10000
Sample keys: _StringKeys(dict_keys(['step_count', 'action', 'observation', 'done', 'terminated', 'truncated', 'next', 'collector', 'action_value', 'chosen_action_value']))
Loss computation time: 0.00s
Replay buffer length: 10000
Iteration: 1
Replay buffer size: 10000
Sample keys: _StringKeys(dict_keys(['step_count', 'action', 'observation', 'done', 'terminated', 'truncated', 'next', 'collector', 'action_value', 'chosen_action_value']))
Loss computation time: 0.00s
Iteration: 2
Replay buffer size: 10000
Sample keys: _StringKeys(dict_keys(['step_count', 'action', 'observation', 'done', 'terminated', 'truncated', 'next', 'collector', 'action_value', 'chosen_action_value']))
Loss computation time: 0.00s
Iteration: 3
Replay buffer size: 10000
Sample keys: _StringKeys(dict_keys(['step_count', 'action', 'observation', 'done', 'terminated', 'truncated', 'next', 'collector', 'action_value', 'chosen_action_value']))
Loss computation time: 0.00s
Iteration: 4

In [ ]:
!apt update && apt install -y ffmpeg
!pip install av



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
54 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
import av
print("PyAV installed successfully!")


PyAV installed successfully!


In [ ]:
!apt update
!apt install -y ffmpeg


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
54 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
!ffmpeg -version


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [ ]:
!ldd $(python -c "import av; print(av.__file__)")


	not a dynamic executable


In [ ]:
!ls ./training

DQN


In [ ]:
record_env.rollout(max_steps=1000 ,policy = epsilon_policy)
video_recorder.dump()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ImportError: PyAV is not installed, and is necessary for the video operations in torchvision.
See https://github.com/mikeboers/PyAV#installation for instructions on how to
install PyAV on your system.
